In [1]:
import pandas as pd
from fbprophet import Prophet
from etl_resources import sqlite_connection
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error
plt.style.use('fivethirtyeight')

In [2]:
def ticker_list():
    
    con = sqlite_connection()
    cur = con.cursor()
    cur.execute(f"select distinct ticker from weekly_prices_clean")
    res = cur.fetchall()
    res = [val[0] for val in res]
    
    return res

In [3]:
def base_df(ticker):
    
    '''
    This function returns the base time series dataframe (date and close)
    '''
    
    con = sqlite_connection()
        
    df = pd.read_sql(f'''select date,close 
    from weekly_prices_clean where ticker='{ticker}' and date>'2017-12-31' 
    group by date,close
    order by date desc''',con=con)
    
    df[['ds', 'y']] = df[['date','close']]
    df = df[['ds', 'y']]
        
    return df

In [4]:
def train_test_split(df, split=0.2):
    
    test_rows = int(round(df.shape[0] * split,0))
    train_rows = df.shape[0] - test_rows
    
    test_df = df.head(test_rows).copy()
    train_df = df.tail(train_rows).copy()
    #print(train_df.shape[0], test_df.shape[0])
    return train_df, test_df

In [5]:
def train_prophet():
    
    tickers = ticker_list()
    
    for ticker in tickers:
        
        print(f"Building model for {ticker}")
        
        # Build the dataset and split
        df = base_df(ticker)
        train, test = train_test_split(df, split=0.1)
        
        # Train & Test Prophet
        model = Prophet(daily_seasonality=True)
        model.fit(train)
        forecast = model.predict(test)
        
        # Create a DF with future dates to predict (weekly candence)
        future = pd.DataFrame()
        future['ds'] = pd.date_range('2018-01-05', periods=250, freq='W-FRI')

        # Make the predictions
        future_fcast = model.predict(future)

        # Plot and Save
        model.plot(future_fcast).savefig(f'../data/visualization/prophet/{ticker}.png')
        
    #return future_fcast


In [ ]:
train_prophet()

Building model for MMM
Initial log joint probability = -4.08416
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       479.455    0.00670291        152.43           1           1      132   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     105        479.85   0.000356316        181.64   2.768e-06       0.001      174  LS failed, Hessian reset 
     199       480.558   4.63672e-06        72.251       0.203      0.6732      293   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     287        481.05    0.00173012       258.489   8.847e-06       0.001      438  LS failed, Hessian reset 
     299       481.141   0.000297055       70.1619      0.6112      0.6112      454   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399       481.341   7.25021e-06       74.4536           1           1      585  

/home/james-laptop/Desktop/RPI_Projects/rensselaer_ai_systems/venv/lib/python3.8/site-packages/fbprophet/plot.py:66: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure(facecolor='w', figsize=figsize)


Building model for CAT
Initial log joint probability = -5.5203
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       453.121    0.00852767       90.0132           1           1      120   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     184       459.205   0.000481481       204.127   3.654e-06       0.001      257  LS failed, Hessian reset 
     199       459.398    0.00031226       108.648      0.7858      0.7858      274   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     241       459.704   0.000217445       79.1771   3.154e-06       0.001      365  LS failed, Hessian reset 
     299       459.733   0.000637409       88.3287      0.3465           1      438   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     300       459.735   0.000168162       79.0491   1.904e-06       0.001      495  L

In [ ]:
#print("Mean Squared Error (MSE):", mean_squared_error(y_true = test["y"], y_pred = forecast['yhat']))
#print("Mean Absolute Error (MAE):", mean_absolute_error(y_true = test["y"], y_pred = forecast['yhat']))

#def mean_abs_perc_err(y_true, y_pred):
#    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

#print("Mean Absolute % Error (MAPE): ", mean_abs_perc_err(y_true = np.asarray(test["y"]), y_pred = np.asarray(forecast['yhat'])))

In [ ]:
#future_fcast
#forecast
model